## Introduction

Since we will implement content-based filtering using a two-tower neural network to build our recommender system, we must create user and video vectors for each interaction in our dataset.

We will only use the interactions from big_matrix to do so, because these represent our train data.

## Imports

In [1]:
import pandas as pd
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## Loading the data

In [2]:
export_dir = "./exports/cleaned_data/"
big_matrix_cleaned = pd.read_parquet(export_dir + "big_matrix_cleaned.pq")
item_categories_cleaned = pd.read_parquet(export_dir + "item_categories_cleaned.pq")
item_daily_features_cleaned = pd.read_parquet(export_dir + "item_daily_features_cleaned.pq")
caption_category_cleaned = pd.read_parquet(export_dir + "caption_category_cleaned.pq")

## Engineering the user vectors

Here is what we want to have in our user vectors to represent each of our users:
- For each video feat, we want to know what is the average watch ratio of the user.
- Same as for video feats, but for the first level caption categories. We want to know what is the average watch ratio of the user for each category.

This will enable us to create a vector representation of our users.

### Step 1: Average watch ratio per video feat

We will explode our originally multi-labelled feature (it is originally a list of feat ids for each video) and compute the average watch ratio for each of this video features, for each one of our users.

In [3]:
def get_user_avg_feat_df(df: pd.DataFrame) -> pd.DataFrame:
    user_avg_feat_df = df[["user_id", "video_id", "watch_ratio"]].copy()
    user_avg_feat_df = user_avg_feat_df.merge(item_categories_cleaned, on="video_id", how="left")
    user_avg_feat_df = user_avg_feat_df.explode(column="feat")
    user_avg_feat_df['avg_feat'] = user_avg_feat_df.groupby(['user_id', 'feat'])['watch_ratio'].transform('mean').fillna(0)
    user_avg_feat_df = user_avg_feat_df[["user_id", "feat", "avg_feat"]].drop_duplicates()

    user_avg_feat_df = user_avg_feat_df.sort_values(["user_id", "feat"]).reset_index(drop=True)
    user_avg_feat_df = user_avg_feat_df.pivot(index='user_id', columns='feat', values='avg_feat')
    user_avg_feat_df.columns = [f'avg_feat_{int(col)}' for col in user_avg_feat_df.columns]
    user_avg_feat_df = user_avg_feat_df.fillna(0)

    return user_avg_feat_df.reset_index()

user_avg_feat_df = get_user_avg_feat_df(big_matrix_cleaned)
user_avg_feat_df

,user_id,avg_feat_0,avg_feat_1,avg_feat_2,avg_feat_3,avg_feat_4,avg_feat_5,avg_feat_6,avg_feat_7,avg_feat_8,...,avg_feat_21,avg_feat_22,avg_feat_23,avg_feat_24,avg_feat_25,avg_feat_26,avg_feat_27,avg_feat_28,avg_feat_29,avg_feat_30
0,0,1.053782,0.921965,0.863959,0.902536,0.762653,1.071888,0.783277,1.021258,1.081992,...,1.092595,0.229430,1.172116,2.434744,0.975717,1.076572,0.824498,1.071799,1.032785,0.485446
1,1,0.365590,0.713516,0.242190,0.183951,1.644048,0.907845,0.823894,0.981698,0.908988,...,1.210433,0.000000,0.102452,0.000000,0.718579,1.090674,2.134150,1.057845,1.434024,1.488989
2,2,0.376160,0.648226,0.481671,0.558891,0.910193,0.664848,0.508172,0.625173,0.665906,...,1.013190,0.000000,0.771147,0.000000,0.605867,0.698716,0.210522,0.656312,0.176768,0.079384
3,3,1.203877,0.917142,0.766411,0.796545,0.991193,1.185631,0.795351,0.959548,1.022053,...,0.897935,0.166655,0.529580,0.000000,0.945197,0.863060,0.623670,1.011033,1.561361,0.524270
4,4,0.000000,0.476605,1.010854,0.000000,0.455023,0.302336,0.488490,0.660835,0.741419,...,0.800958,0.000000,0.000000,0.000000,0.962420,0.963989,0.000000,0.895585,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6894,7171,0.603449,0.841624,0.446053,0.526626,0.717613,0.767264,0.602171,0.822128,0.760489,...,0.279137,0.019471,0.416539,0.000000,0.814844,0.819282,0.846472,0.938140,0.181914,0.681021
6895,7172,0.700809,0.968383,0.716270,0.377055,0.615240,0.902377,0.938824,0.932746,1.146590,...,0.989900,0.058764,1.305065,0.000000,0.867869,1.092383,0.947174,1.110529,0.000000,0.522903
6896,7173,0.293560,0.255835,1.569094,0.000000,0.365562,0.462788,0.673106,0.600301,0.609157,...,0.000000,0.000000,0.441495,0.000000,0.770451,0.771188,0.000000,0.817329,0.000000,0.585139
6897,7174,1.071403,0.710077,0.524371,0.767773,0.932450,0.845830,0.631634,0.832533,0.856360,...,0.680289,0.083650,3.462550,0.000000,0.810274,0.775306,0.712956,0.926372,0.831416,0.772563


### Step 2: Average watch ratio per video caption category

We will perform the same process as for the Step 1 in order to compute the average watch ratio of the user for each first level caption category a video might have. 

In [4]:
def get_user_avg_category_df(df: pd.DataFrame) -> pd.DataFrame:
    user_avg_category_df = df.copy()
    user_avg_category_df = user_avg_category_df.merge(caption_category_cleaned, on="video_id", how="left")
    user_avg_category_df = user_avg_category_df.explode(column="first_level_category_id")
    user_avg_category_df['avg_category'] = user_avg_category_df.groupby(['user_id', 'first_level_category_id'])['watch_ratio'].transform('mean').fillna(0)
    user_avg_category_df = user_avg_category_df[["user_id", "first_level_category_id", "avg_category"]].drop_duplicates()

    user_avg_category_df = user_avg_category_df.sort_values(["user_id", "first_level_category_id"]).reset_index(drop=True)
    user_avg_category_df = user_avg_category_df.pivot(index='user_id', columns='first_level_category_id', values='avg_category')
    user_avg_category_df.columns = [f'avg_category_{int(col)}' for col in user_avg_category_df.columns]
    user_avg_category_df = user_avg_category_df.fillna(0)

    return user_avg_category_df.reset_index()

user_avg_category_df = get_user_avg_category_df(big_matrix_cleaned)
user_avg_category_df

,user_id,avg_category_1,avg_category_2,avg_category_3,avg_category_4,avg_category_5,avg_category_6,avg_category_7,avg_category_8,avg_category_9,...,avg_category_30,avg_category_31,avg_category_32,avg_category_33,avg_category_34,avg_category_35,avg_category_36,avg_category_37,avg_category_38,avg_category_39
0,0,1.004131,0.824647,0.949826,0.921281,1.040737,0.821951,1.059865,1.180643,1.007497,...,1.410021,1.003802,0.784949,1.082833,0.937071,1.023252,0.876753,0.836855,0.586343,1.098977
1,1,0.769762,0.269092,0.053281,0.993688,0.946753,0.960627,1.020950,1.007514,0.912520,...,1.234668,1.029305,0.511845,0.587909,0.650246,1.091391,0.897980,1.489968,3.278239,0.235781
2,2,0.658760,0.345140,0.558891,0.909273,0.670960,0.739852,0.672275,0.767115,0.618431,...,0.875462,0.539263,0.455565,0.678805,0.563569,0.581113,0.464605,0.446583,0.000000,0.000000
3,3,0.908873,0.722965,0.788544,1.005447,1.108424,0.843249,0.905177,1.047106,0.891439,...,1.032950,1.135903,0.465189,1.037848,1.076418,0.934153,0.576851,0.550652,0.756021,1.022186
4,4,0.454793,1.182808,0.000000,0.455023,0.422407,0.405792,0.844153,0.742764,1.067094,...,0.474285,0.354229,0.324213,0.864947,0.748424,0.411726,0.000000,0.290513,0.000000,0.060355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6894,7171,0.869806,0.212285,0.334050,0.940574,0.837207,0.772427,0.898251,0.754787,0.742373,...,0.480705,0.600195,0.512077,1.020102,0.719907,1.052949,0.168634,0.207517,0.000000,0.038046
6895,7172,1.035662,0.473096,0.382845,0.745258,1.006997,1.107974,0.863858,1.186230,0.878089,...,0.515929,0.998493,0.804615,0.985133,0.954153,1.672876,1.032393,0.748696,0.813821,0.206309
6896,7173,0.326512,0.261622,0.000000,0.477246,0.428633,0.861025,0.855456,0.252676,0.504648,...,0.000000,0.482575,0.327401,2.141854,0.785324,0.394952,0.559333,0.196024,0.309758,0.019076
6897,7174,0.725369,0.523844,0.858081,1.048326,0.835873,0.705095,0.820545,0.928850,0.785818,...,0.976515,0.883538,0.317697,0.958845,0.780613,0.800540,0.247095,0.609666,2.318234,0.438914


### Step 3: Merge everything to create our user vectors

In [5]:
user_df = user_avg_feat_df.copy().merge(user_avg_category_df, on="user_id", how="left")
user_df

,user_id,avg_feat_0,avg_feat_1,avg_feat_2,avg_feat_3,avg_feat_4,avg_feat_5,avg_feat_6,avg_feat_7,avg_feat_8,...,avg_category_30,avg_category_31,avg_category_32,avg_category_33,avg_category_34,avg_category_35,avg_category_36,avg_category_37,avg_category_38,avg_category_39
0,0,1.053782,0.921965,0.863959,0.902536,0.762653,1.071888,0.783277,1.021258,1.081992,...,1.410021,1.003802,0.784949,1.082833,0.937071,1.023252,0.876753,0.836855,0.586343,1.098977
1,1,0.365590,0.713516,0.242190,0.183951,1.644048,0.907845,0.823894,0.981698,0.908988,...,1.234668,1.029305,0.511845,0.587909,0.650246,1.091391,0.897980,1.489968,3.278239,0.235781
2,2,0.376160,0.648226,0.481671,0.558891,0.910193,0.664848,0.508172,0.625173,0.665906,...,0.875462,0.539263,0.455565,0.678805,0.563569,0.581113,0.464605,0.446583,0.000000,0.000000
3,3,1.203877,0.917142,0.766411,0.796545,0.991193,1.185631,0.795351,0.959548,1.022053,...,1.032950,1.135903,0.465189,1.037848,1.076418,0.934153,0.576851,0.550652,0.756021,1.022186
4,4,0.000000,0.476605,1.010854,0.000000,0.455023,0.302336,0.488490,0.660835,0.741419,...,0.474285,0.354229,0.324213,0.864947,0.748424,0.411726,0.000000,0.290513,0.000000,0.060355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6894,7171,0.603449,0.841624,0.446053,0.526626,0.717613,0.767264,0.602171,0.822128,0.760489,...,0.480705,0.600195,0.512077,1.020102,0.719907,1.052949,0.168634,0.207517,0.000000,0.038046
6895,7172,0.700809,0.968383,0.716270,0.377055,0.615240,0.902377,0.938824,0.932746,1.146590,...,0.515929,0.998493,0.804615,0.985133,0.954153,1.672876,1.032393,0.748696,0.813821,0.206309
6896,7173,0.293560,0.255835,1.569094,0.000000,0.365562,0.462788,0.673106,0.600301,0.609157,...,0.000000,0.482575,0.327401,2.141854,0.785324,0.394952,0.559333,0.196024,0.309758,0.019076
6897,7174,1.071403,0.710077,0.524371,0.767773,0.932450,0.845830,0.631634,0.832533,0.856360,...,0.976515,0.883538,0.317697,0.958845,0.780613,0.800540,0.247095,0.609666,2.318234,0.438914


We now successfuly created a vector representation for each of our unique 6899 users, containing each 70 engineered features.

## Engineering the video vectors

The video vectors are a bit more complicated to engineer because we want:
- A vector encoding for each video feat (1 if the video has it, 0 otherwise)
- A vector encoding for each first level caption category id (1 if the video's caption has this id, 0 otherwise)
- The video duration
- A trending score

The trending score is the hard part, because we will compute a trend score for the last 7 days rolling prior to the interaction time. This means each video will have different vector representations depending on the time of the interaction.

### Step 1: Video feat vector encoding

In [6]:
def get_video_feat_df(df: pd.DataFrame) -> pd.DataFrame:
    video_feat_df = df.copy()
    video_feat_df = video_feat_df.explode(column="feat")
    video_feat_df = pd.crosstab(video_feat_df["video_id"], video_feat_df["feat"])
    video_feat_df.columns = [f'feat_{int(col)}' for col in video_feat_df.columns]
    return video_feat_df

video_feat_df = get_video_feat_df(item_categories_cleaned)
video_feat_df

,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30
video_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10722,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10723,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10724,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Step 2: Video caption first level category vector encoding 

In [7]:
def get_video_category_df(df: pd.DataFrame) -> pd.DataFrame:
    video_category_df = df.copy()
    video_category_df = pd.crosstab(video_category_df["video_id"], video_category_df["first_level_category_id"])
    video_category_df.columns = [f'category_{int(col)}' for col in video_category_df.columns]
    return video_category_df

video_category_df = get_video_category_df(caption_category_cleaned)
video_category_df

,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8,category_9,category_10,...,category_30,category_31,category_32,category_33,category_34,category_35,category_36,category_37,category_38,category_39
video_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10722,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10723,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
10724,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Step 3: Video duration and 7 days rolling trend score

In order to compute our 7 days rolling trend score, we will:
- Compute a weighted engagement score for each video based on interactions nature and count (for example, we weighted the score in order to give more value to a follower count increase than to a like).
- Apply a log10 to reduce skew.
- Fill in missing dates so we have data for each day for each video. Originally, we do not have an entry every day for each video, so we add some while assuming the video had no interactions for the missing days.
- Mask the scores before the upload date, so we do not dilude our engagement score just after the date of a video upload.
- Compute rolling the 7 days rolling trend score as a mean of the last 7 calendar days (not including the current day).

For the video duration feature, we just have to fill the value in the days we added.

In [8]:
def compute_engagement_score(df: pd.DataFrame) -> pd.DataFrame:
    weights = {"valid_play_cnt": 0.1, "like_cnt": 0.2, "comment_cnt": 0.3, "share_cnt": 0.5, "follow_cnt": 1.0, "collect_cnt": 0.5, "download_cnt": 0.5}
    df["score"] = sum(df[col] * weight for col, weight in weights.items())
    df.drop(columns=weights.keys(), inplace=True)
    df["score"] = np.log10(df["score"])
    df["score"].replace(-np.inf, np.nan, inplace=True)
    return df

def fill_missing_dates(df: pd.DataFrame, start="2020-06-23", end="2020-09-10") -> pd.DataFrame:
    full_index = pd.MultiIndex.from_product([df["video_id"].unique(), pd.date_range(start, end)], names=["video_id", "date"])
    df = df.set_index(["video_id", "date"]).reindex(full_index).reset_index()
    return df

def mask_score_before_upload(df: pd.DataFrame) -> pd.DataFrame:
    df["upload_dt"] = df.groupby("video_id")["upload_dt"].transform(lambda x: x.ffill().bfill())
    df["score_masked"] = df.apply(lambda row: row["score"] if row["date"] >= row["upload_dt"] else np.nan, axis=1)
    df["score"] = df["score"].fillna(0)
    return df

def compute_rolling_trend(group):
    group = group.sort_values("date").copy()
    upload_dt = group["upload_dt"].iloc[0]

    group = group.set_index("date")
    valid = group.index > upload_dt
    trend = pd.Series(np.nan, index=group.index)

    trend[valid] = group["score_masked"][valid].shift(1).rolling("7D", min_periods=1).mean()
    group["trend_score"] = trend

    return group.reset_index()[["video_id", "date", "trend_score"]]

def compute_trend(df: pd.DataFrame) -> pd.DataFrame:
    trend_scores = df.groupby("video_id", group_keys=False).apply(compute_rolling_trend).reset_index(drop=True).fillna(0)
    return trend_scores

def finalize_df(df, trend_scores):
    df = pd.merge(df, trend_scores, on=["video_id", "date"], how="left")
    df["video_duration"] = df.groupby("video_id")["video_duration"].transform(lambda x: x.ffill().bfill())
    df = df.drop(columns=["upload_dt", "score", "score_masked"])
    df = df.sort_values(["video_id", "date"])
    return df

def process_video_dataframe() -> pd.DataFrame:
    df = item_daily_features_cleaned.copy()
    df = compute_engagement_score(item_daily_features_cleaned)
    df = fill_missing_dates(df)
    df = mask_score_before_upload(df)
    trend_scores = compute_trend(df)
    df = finalize_df(df, trend_scores)
    return df

video_df = process_video_dataframe()
video_df

,video_id,date,video_duration,trend_score
0,0,2020-06-23,5966.0,0.0
1,0,2020-06-24,5966.0,0.0
2,0,2020-06-25,5966.0,0.0
3,0,2020-06-26,5966.0,0.0
4,0,2020-06-27,5966.0,0.0
...,...,...,...,...
700315,10727,2020-09-06,5666.0,0.0
700316,10727,2020-09-07,5666.0,0.0
700317,10727,2020-09-08,5666.0,0.0
700318,10727,2020-09-09,5666.0,0.0


### Step 4: Merge everything to create our video vectors

In [9]:
video_df = video_df.merge(video_feat_df, on="video_id", how="left")
video_df = video_df.merge(video_category_df, on="video_id", how="left")
video_df

,video_id,date,video_duration,trend_score,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,...,category_30,category_31,category_32,category_33,category_34,category_35,category_36,category_37,category_38,category_39
0,0,2020-06-23,5966.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2020-06-24,5966.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2020-06-25,5966.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2020-06-26,5966.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,2020-06-27,5966.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700315,10727,2020-09-06,5666.0,0.0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
700316,10727,2020-09-07,5666.0,0.0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
700317,10727,2020-09-08,5666.0,0.0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
700318,10727,2020-09-09,5666.0,0.0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


We now successfuly created a vector representation for each of our videos and so for each possible day in our dataset.

## Saving the dataframes

In [10]:
export_dir = "./exports/feature_engineered_data/"
if not os.path.exists(export_dir):
    os.makedirs(export_dir)
user_df.to_parquet(export_dir + "user_df.pq")
video_df.to_parquet(export_dir + "video_df.pq")